<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e42857ade1e5796923868f0cedabb1abff1ff341be1e8b4fa5b975b3af12b990
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-04 11:02:56
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.45 C
-------------------
Today PnL: -28.09 K (-0.19%)
Current PnL: -19.41 L (-12.72%)
CY Booked + Current PnL: -6.76 L (-4.43%)
-------------------
Total profit:  2.33 L
Total loss:  -21.74 L
-------------------
Total Booked + Current PnL: 20.22 L (16.09%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.72%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 86.40 L (59.52%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.52%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.28,67.0,X-LC,5.28,231613.0,21247.0,7527.0,-0.11,10.10,3.25,13.68,37.0,2.82,1.61,28.29,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,105.76,57.0,M-SC,1.74,88687.0,-12090.0,12186.0,-0.25,-12.00,13.74,0.09,245.0,-0.99,0.62,16.52,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-5.43,67.0,M-LC,10.16,171687.0,30847.0,13649.0,0.08,21.90,7.95,31.59,66.0,2.26,1.19,54.19,XY24,NTT,BANKS
33,ICICIGI,2252.93,-12.22,65.0,X-MC,5.27,151101.0,14968.0,15609.0,1.71,10.99,10.33,22.47,91.0,0.96,1.05,25.40,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-10.45,41.0,X-MC,2.49,88395.0,-2283.0,17193.0,1.48,-2.52,19.45,16.44,101.0,-0.13,0.61,17.15,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-51.56,78.0,H-SC,6.80,279521.0,-6984.0,264315.0,3.23,-2.44,94.56,89.82,134.0,-0.03,1.94,47.89,XY24,NTT,FINANCE
33,ICICIGI,2252.93,-12.22,65.0,X-MC,5.27,151101.0,14968.0,15609.0,1.71,10.99,10.33,22.47,91.0,0.96,1.05,25.40,X40,ATH,INSURANCE
74,TATAMOTORS,1065.00,-51.85,23.0,X-LC,26.39,151371.0,-125152.0,235231.0,1.71,-45.26,155.40,39.81,54.0,-0.53,1.05,6.69,XY24,NTT,AUTO
82,VALIANTORG,838.00,-289.64,46.0,H-SC,10.56,105400.0,-58205.0,179517.0,1.62,-35.58,170.32,74.15,139.0,-0.32,0.73,35.11,XR,NTT,CHEMICALS
2,ABBOTINDIA,35195.00,-10.45,41.0,X-MC,2.49,88395.0,-2283.0,17193.0,1.48,-2.52,19.45,16.44,101.0,-0.13,0.61,17.15,X40,ATH,PHARMA


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TITAGARH,1548.00,-4.95,49.0,H-SC,3.63,202931.0,-39246.0,150007.0,-1.86,-16.21,73.92,45.74,158.0,-0.26,1.41,31.99,XY24,NTT,ENGINEERING
49,MASFIN,398.61,-16.89,58.0,H-SC,8.81,94665.0,-3315.0,24916.0,-1.45,-3.38,26.32,22.05,152.0,-0.13,0.66,37.60,XR,ATH,FINANCE
31,HINDZINC,730.22,26.60,46.0,M-LC,10.69,206125.0,1049.0,111514.0,-1.41,0.51,54.10,54.89,52.0,0.01,1.43,24.73,X5K,ATH,METALS
55,RAJOOENG,143.10,-42.97,44.0,H-SC,17.02,86460.0,-16040.0,56640.0,-1.37,-15.65,65.51,39.61,114.0,-0.28,0.60,5.59,AR,ATH,MISC
3,ACC,3906.00,-38.07,47.0,X-MC,2.44,184500.0,-53451.0,206105.0,-1.36,-22.46,111.71,64.15,174.0,-0.26,1.28,3.49,XY24,BTT,CEMENT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,26.6,46.0,M-LC,10.69,206125.0,1049.0,111514.0,-1.41,0.51,54.1,54.89,52.0,0.01,1.43,24.73,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,105.76,57.0,M-SC,1.74,88687.0,-12090.0,12186.0,-0.25,-12.00,13.74,0.09,245.0,-0.99,0.62,16.52,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.63,221211.0,-48456.0,84790.0,-1.35,-17.97,38.33,13.47,138.0,-0.57,1.54,12.48,XY24,NTT,PAINTS
17,CERA,9475.0,-21.72,54.0,H-SC,1.96,142750.0,-33153.0,75172.0,-1.22,-18.85,52.66,23.89,149.0,-0.44,0.99,23.28,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-24.05,45.0,H-MC,6.03,102695.0,-29140.0,71363.0,-0.53,-22.10,69.49,32.03,98.0,-0.41,0.71,15.03,OX40N,NTT,IT
67,SIS,528.0,2026.94,46.0,H-SC,2.98,85833.0,-25199.0,48805.0,1.05,-22.70,56.86,21.26,156.0,-0.52,0.60,15.97,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,67.0,M-LC,10.16,171687.0,30847.0,13649.0,0.08,21.9,7.95,31.59,66.0,2.26,1.19,54.19,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,51.56,62.0,M-MC,9.78,238876.0,13914.0,153741.0,0.54,6.19,64.36,74.53,192.0,0.09,1.66,40.43,XY24,BTT,STEEL
56,RECLTD,446.00,47.87,53.0,M-LC,4.94,206965.0,3795.0,38330.0,-0.76,1.87,18.52,20.74,55.0,0.10,1.44,8.86,XY25,NTT,FINANCE
31,HINDZINC,730.22,26.60,46.0,M-LC,10.69,206125.0,1049.0,111514.0,-1.41,0.51,54.10,54.89,52.0,0.01,1.43,24.73,X5K,ATH,METALS
46,KPIGREEN,731.05,13.63,75.0,H-SC,6.55,135311.0,10014.0,48915.0,-0.34,7.99,36.15,47.03,141.0,0.20,0.94,69.42,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,67.0,M-LC,10.16,171687.0,30847.0,13649.0,0.08,21.90,7.95,31.59,66.0,2.26,1.19,54.19,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,13.63,75.0,H-SC,6.55,135311.0,10014.0,48915.0,-0.34,7.99,36.15,47.03,141.0,0.20,0.94,69.42,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-39.15,58.0,M-SC,4.63,126141.0,11343.0,82698.0,-0.52,9.88,65.56,81.92,223.0,0.14,0.88,49.33,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,6.07,130645.0,7309.0,124322.0,-0.07,5.93,95.16,106.72,119.0,0.06,0.91,30.69,AR,ATH,MISC
31,HINDZINC,730.22,26.60,46.0,M-LC,10.69,206125.0,1049.0,111514.0,-1.41,0.51,54.10,54.89,52.0,0.01,1.43,24.73,X5K,ATH,METALS
86,WIPRO,420.00,-15.61,41.0,M-LC,5.38,148453.0,-2492.0,111948.0,-0.44,-1.65,75.41,72.51,53.0,-0.02,1.03,4.27,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-51.85,23.0,X-LC,26.39,151371.0,-125152.0,235231.0,1.71,-45.26,155.40,39.81,54.0,-0.53,1.05,6.69,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,6.32,88868.0,-39802.0,85100.0,0.02,-30.93,95.76,35.20,219.0,-0.47,0.62,2.00,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-21.71,34.0,X-MC,4.84,196290.0,-4057.0,34567.0,0.29,-2.02,17.61,15.23,178.0,-0.12,1.36,29.57,X40N,NTT,REALTY
30,HINDUNILVR,2922.00,-12.79,35.0,X-LC,7.02,244900.0,-12163.0,47290.0,-0.45,-4.73,19.31,13.67,24.0,-0.26,1.70,14.71,XY25,NTT,FMCG
23,DMART,5391.45,-17.64,38.0,X-LC,8.66,209880.0,-527.0,59690.0,1.00,-0.25,28.44,28.12,38.0,-0.01,1.46,23.32,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-37.50,52.0,X-LC,1.15,201835.0,1697.0,19195.0,-0.29,0.85,9.51,10.44,4.0,0.09,1.40,6.46,X40,NTT,FMCG
66,SIEMENS,4671.50,-2.83,47.0,H-LC,1.24,156145.0,-29950.0,77432.0,-0.15,-16.09,49.59,25.51,15.0,-0.39,1.08,15.07,AR,ATH,ELECTRICAL
52,PGHH,17907.65,-30.55,39.0,X-MC,1.28,200850.0,30.0,67767.0,-0.01,0.01,33.74,33.76,80.0,0.00,1.39,5.21,X40,ATH,FMCG
28,HAVELLS,2069.16,-6.32,45.0,X-MC,2.90,312690.0,-15301.0,121824.0,-0.55,-4.66,38.96,32.48,92.0,-0.13,2.17,6.23,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-4.67,75.0,X-LC,4.25,249933.0,15789.0,32841.0,0.38,6.74,13.14,20.77,86.0,0.48,1.73,14.13,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,39.0,X-SC,37.71,51182.0,-13824.0,164750.0,-0.73,-21.27,321.89,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,6.32,88868.0,-39802.0,85100.0,0.02,-30.93,95.76,35.20,219.0,-0.47,0.62,2.00,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-28.27,38.0,X-MC,7.05,81100.0,-2410.0,22440.0,-0.52,-2.89,27.67,23.98,82.0,-0.11,0.56,2.77,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.08,44.0,X-MC,7.34,219510.0,-43855.0,153174.0,-0.24,-16.65,69.78,41.51,84.0,-0.29,1.52,3.01,XY25,ATH,FMCG
3,ACC,3906.00,-38.07,47.0,X-MC,2.44,184500.0,-53451.0,206105.0,-1.36,-22.46,111.71,64.15,174.0,-0.26,1.28,3.49,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,39.0,X-SC,37.71,51182.0,-13824.0,164750.0,-0.73,-21.27,321.89,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.27,38.0,X-MC,7.05,81100.0,-2410.0,22440.0,-0.52,-2.89,27.67,23.98,82.0,-0.11,0.56,2.77,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.17,42.0,X-SC,4.45,81583.0,-63577.0,139507.0,-0.11,-43.80,171.00,52.31,136.0,-0.46,0.57,9.20,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.45,41.0,X-MC,2.49,88395.0,-2283.0,17193.0,1.48,-2.52,19.45,16.44,101.0,-0.13,0.61,17.15,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,6.32,88868.0,-39802.0,85100.0,0.02,-30.93,95.76,35.20,219.0,-0.47,0.62,2.00,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.69,49.0,X-LC,10.23,283579.0,-62377.0,129085.0,-1.35,-18.03,45.52,19.28,1.0,-0.48,1.97,4.83,X40,ATH,IT
39,INFY,2275.00,-19.24,50.0,X-LC,5.24,314601.0,1713.0,169979.0,-0.57,0.55,54.03,54.87,3.0,0.01,2.18,8.88,X40,BTT,IT
41,ITC,452.00,-37.50,52.0,X-LC,1.15,201835.0,1697.0,19195.0,-0.29,0.85,9.51,10.44,4.0,0.09,1.40,6.46,X40,NTT,FMCG
83,VBL,671.64,-16.73,53.0,X-LC,7.20,300115.0,-15727.0,128389.0,-0.92,-4.98,42.78,35.67,5.0,-0.12,2.08,8.19,X40N,ATH,FMCG
1,ABB,7934.00,-39.67,53.0,H-LC,5.22,252024.0,-9595.0,128809.0,-0.10,-3.67,51.11,45.57,7.0,-0.07,1.75,7.24,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,43.0,L-SC,28.09,78720.0,-34829.0,74878.0,-0.63,-30.67,95.12,35.27,268.0,-0.47,0.55,92.67,XR,NTT,HOTELS
7,ASIANTILES,137.00,7394.44,68.0,L-SC,9.77,82228.0,-11582.0,88198.0,0.06,-12.35,107.26,81.67,269.0,-0.13,0.57,59.24,XR,NTT,CERAMICS
49,MASFIN,398.61,-16.89,58.0,H-SC,8.81,94665.0,-3315.0,24916.0,-1.45,-3.38,26.32,22.05,152.0,-0.13,0.66,37.60,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-39.15,58.0,M-SC,4.63,126141.0,11343.0,82698.0,-0.52,9.88,65.56,81.92,223.0,0.14,0.88,49.33,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,6.07,130645.0,7309.0,124322.0,-0.07,5.93,95.16,106.72,119.0,0.06,0.91,30.69,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,81.27,73.0,H-SC,1.77,157243.0,-25532.0,136597.0,-0.87,-13.97,86.87,60.77,125.0,-0.19,1.09,40.84,XR,NTT,JEWELLERY
59,REPCOHOME,880.00,-51.56,78.0,H-SC,6.80,279521.0,-6984.0,264315.0,3.23,-2.44,94.56,89.82,134.0,-0.03,1.94,47.89,XY24,NTT,FINANCE
46,KPIGREEN,731.05,13.63,75.0,H-SC,6.55,135311.0,10014.0,48915.0,-0.34,7.99,36.15,47.03,141.0,0.20,0.94,69.42,MH,ATH,POWER
62,SAMMAANCAP,326.00,-198.86,77.0,M-SC,32.34,170991.0,21771.0,122412.0,0.03,14.59,71.59,96.62,208.0,0.18,1.19,85.05,XY25,NTT,FINANCE
68,SONACOMS,806.63,-31.80,67.0,M-SC,7.24,84928.0,-16231.0,56231.0,0.84,-16.04,66.21,39.54,202.0,-0.29,0.59,19.76,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.65
1,25,44.05
2,50,75.49


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.18
LC    32.90
MC    22.92
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.02
X40      16.10
X40N     12.47
XY25     11.91
XR        9.94
AR        7.82
OX40N     6.54
X200      1.76
X5K       1.43
SR        1.07
MH        0.94
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.43
X-LC    22.25
X-MC    18.61
M-SC    12.73
X-SC     5.57
M-LC     5.09
H-LC     4.59
H-MC     2.34
M-MC     1.66
L-SC     1.45
L-LC     0.97
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.12,-3.83,37.39
IT,12.37,-17.85,80.99
FINANCE,12.27,-7.07,55.60
MISC,7.82,-15.52,73.78
ELECTRICAL,5.77,-8.59,47.09
PAINTS,5.45,-17.63,35.17
BANKS,4.04,-15.88,83.29
INSURANCE,3.84,0.08,37.17
AUTO,2.78,-43.51,103.41


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3249516.0,22
XR,1223594.0,13
AR,1160161.0,9
X40,835148.0,12
X40N,672779.0,9
OX40N,542897.0,9
XY25,457922.0,8
SR,267106.0,2
X5K,111514.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3311750.0,24
M-SC,1211675.0,15
X-MC,1190858.0,14
X-LC,1071919.0,13
X-SC,549198.0,6
H-LC,276518.0,3
M-LC,275441.0,4
L-SC,252197.0,3
H-MC,250033.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1161583.0      6
           AR         831659.0      5
           XR         760973.0      7
M-SC       XY24       695907.0      6
X-MC       XY24       575561.0      4
X-LC       X40        463570.0      5
           XY24       305655.0      2
X-SC       X40N       299348.0      4
X-MC       X40        286478.0      6
H-SC       SR         267106.0      2
           OX40N      241514.0      3
X-LC       X40N       220920.0      3
H-LC       AR         206241.0      2
H-MC       XY24       178670.0      1
X-MC       XY25       176308.0      2
X-SC       XY24       164750.0      1
L-SC       XR         163076.0      2
M-MC       XY24       153741.0      1
X-MC       X40N       152511.0      2
M-SC       OX40N      140899.0      4
           XR         130196.0      2
           XY25       122412.0      1
           AR         122261.0      2
M-LC       XR         111948.0      1
           X5K        111514.0      1
L-SC       OX40N       89121.0      1
X-SC       X40         85100.0      1
X-LC       XY25        81774.0      3
H-MC       OX40N       71363.0      1
H-LC       X200        70277.0      1
L-MC       XR          57401.0      1
H-SC       MH          48915.0      1
L-LC       XY25        39098.0      1
M-LC       XY25        38330.0      1
           XY24        13649.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
